<a href="https://colab.research.google.com/github/cdbm/airbnb-price-rediction-optimizer/blob/main/airbnb_predictorV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd

# Database pre processing

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/cdbm/airbnb-price-rediction-optimizer/main/AB_NYC_2019.csv")
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [ ]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [ ]:
df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [ ]:
df = df.drop('name', axis=1)
df = df.drop('host_name', axis=1)
df = df.drop('last_review', axis=1)

In [ ]:
df = df[df.price <= 500]

In [ ]:
# df.loc[df['price'] <= 50, 'ranges'] = '0_TO_50'
df.loc[df['price'] <= 100, 'ranges'] = '0_TO_100'
df.loc[(df['price'] > 100) & (df['price'] <= 200), 'ranges'] = '101_TO_200'
# df.loc[(df['price'] > 150) & (df['price'] <= 200), 'ranges'] = '151_TO_200'
df.loc[(df['price'] > 200) & (df['price'] <= 300), 'ranges'] = '201_TO_300'
#df.loc[(df['price'] > 250) & (df['price'] <= 300), 'ranges'] = '251_TO_300'
df.loc[(df['price'] > 300) & (df['price'] <= 400), 'ranges'] = '301_TO_400'
#df.loc[(df['price'] > 350) & (df['price'] <= 400), 'ranges'] = '351_TO_400'
df.loc[(df['price'] > 400) & (df['price'] <= 500), 'ranges'] = '401_TO_500'
#df.loc[(df['price'] > 450) & (df['price'] <= 500), 'ranges'] = '551_TO_500'

In [ ]:
df['neighbourhood_group'] = df['neighbourhood_group'].astype('category')
df['neighbourhood'] = df['neighbourhood'].astype('category')
df['room_type'] = df['room_type'].astype('category')

In [ ]:
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

In [ ]:
df['ranges'] = df['ranges'].astype('category')

In [ ]:
df['reviews_per_month'].fillna(0, inplace=True) 

In [ ]:
df = df.drop('price', axis=1)


In [ ]:
df.head()

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,ranges
0,2539,2787,1,107,40.64749,-73.97237,1,1,9,0.21,6,365,101_TO_200
1,2595,2845,2,126,40.75362,-73.98377,0,1,45,0.38,2,355,201_TO_300
2,3647,4632,2,93,40.80902,-73.94190,1,3,0,0.00,1,365,101_TO_200
3,3831,4869,1,41,40.68514,-73.95976,0,1,270,4.64,1,194,0_TO_100
4,5022,7192,2,61,40.79851,-73.94399,0,10,9,0.10,1,0,0_TO_100


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df = df[:10000]
df.describe()

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.896925e+07,6.787538e+07,1.652900,106.624000,40.728702,-73.951510,0.509700,6.922500,23.575700,1.076120,7.254900,110.960200
std,1.088128e+07,7.857125e+07,0.735986,68.723943,0.054700,0.045742,0.537713,21.829809,45.366958,1.529367,34.102296,130.630512
min,8.110000e+03,2.845000e+03,0.000000,0.000000,40.539820,-74.176280,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,9.492466e+06,7.930734e+06,1.000000,51.000000,40.689690,-73.982382,0.000000,1.000000,1.000000,0.050000,1.000000,0.000000
50%,1.959412e+07,3.167435e+07,2.000000,93.000000,40.722350,-73.955130,0.000000,2.000000,5.000000,0.380000,1.000000,42.000000
75%,2.894009e+07,1.074344e+08,2.000000,180.000000,40.763040,-73.936752,1.000000,5.000000,24.000000,1.580000,2.000000,219.000000
max,3.648543e+07,2.743076e+08,4.000000,218.000000,40.911690,-73.725820,2.000000,1000.000000,607.000000,19.750000,327.000000,365.000000


In [ ]:
df

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,ranges
0,18946416,19867664,1,51,40.67100,-73.94703,0,1,105,4.57,2,274,101_TO_200
1,30507545,201447930,2,64,40.72646,-73.98686,0,2,2,0.32,2,28,201_TO_300
2,34142272,257773717,1,213,40.71041,-73.96327,1,30,0,0.00,5,341,0_TO_100
3,27054483,12275644,1,84,40.66774,-73.99292,1,2,0,0.00,1,0,0_TO_100
4,21067451,49108760,2,94,40.76195,-73.99077,1,1,4,0.22,1,0,201_TO_300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,30389608,107434423,2,64,40.73169,-73.98932,0,30,0,0.00,232,157,401_TO_500
9996,15377806,8903368,2,93,40.81668,-73.94407,1,1,3,0.09,1,0,0_TO_100
9997,66741,327673,1,150,40.67732,-73.98225,0,2,214,2.08,2,263,101_TO_200
9998,29149612,21030441,3,4,40.76836,-73.91506,1,2,20,3.24,1,282,0_TO_100


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [ ]:
  features = df.drop('ranges',axis=1)
  labels = df['ranges']

In [ ]:
feature_train, feature_test, label_train, label_test = train_test_split(features, labels, test_size=0.2, random_state=1)

In [ ]:
sc_X = StandardScaler()
feature_train=sc_X.fit_transform(feature_train)
feature_test=sc_X.fit_transform(feature_test)

# Genetic Algorithm

In [ ]:
import random

## Parâmetros & Função Fitness


In [ ]:
POPULATION_SIZE = 30
GENERATIONS = 100
CHILDREN_RATE = 3

In [ ]:
activation = ['identity', 'logistic', 'tanh', 'relu']
solver = ['lbfgs','sgd', 'adam']
learning_rate = ['constant', 'invscaling', 'adaptive']

In [ ]:
def calculate_fitness(individue):
  model_mlp = MLPClassifier(activation=individue['activation'],
                            hidden_layer_sizes=individue['hidden_layer_sizes'],
                            learning_rate=individue['learning_rate'],
                            max_iter=individue['max_iter'],
                            solver=individue['solver'],
                            shuffle=individue['shuffle'],
                            warm_start=individue['warm_start'],
                            momentum=individue['momentum'],
                            nesterovs_momentum=individue['nesterovs_momentum'],
                            batch_size=individue['batch_size'])
  

  model_mlp.fit(feature_train, label_train) 

  prediction = model_mlp.predict(feature_test) 
  return accuracy_score(label_test, prediction)
  

## Geração Ppulação 


In [ ]:
def generate_individue():
  individue = {
      'activation':  activation[random.randint(0, 3)],
      'hidden_layer_sizes' : (random.randint(100, 200), random.randint(50, 100) ,random.randint(10, 50)),
      'learning_rate': learning_rate[random.randint(0,2)],
      'max_iter' : random.randint(100, 250),
      'solver': solver[random.randint(0,2)],
      'shuffle' : (random.randint(0,1) == 1),
      'warm_start' : (random.randint(0,1) == 1),
      'momentum' : random.randrange(0,1),
      'nesterovs_momentum' : (random.randint(0,1) == 1),
      'batch_size' : random.randint(100,300),
      'fitness': 0  
  }

  individue['fitness'] = calculate_fitness(individue)
  return individue

In [ ]:
def sortByFitness(e):
  return e['fitness']

In [ ]:
def initiate_population(population_size):
  population = [[]] * population_size
  population = [generate_individue() for i in range(0, population_size)]
  population.sort(key=sortByFitness, reverse=True)
  return population

## Mutações


In [ ]:
def choose_parents(population):
  father = random.randint(0, POPULATION_SIZE - 1)
  mother = random.randint(0, POPULATION_SIZE - 1)
  return [population[father], population[mother]]

In [ ]:
def choose_candidates(population):
  candidatesIndex = []
  for x in range(0, round(POPULATION_SIZE * 0.3) ):
    index = random.randint(0, POPULATION_SIZE - 1)
    while index in candidatesIndex:
      index = random.randint(0, POPULATION_SIZE - 1)
    candidatesIndex.append(index)

  candidates = []
  for x in candidatesIndex:
    candidates.append(population[x])
  return candidates   

def choose_parents_by_roullete(population):
  population.sort(key=sortByFitness, reverse=True)
  candidates = population[:round(POPULATION_SIZE / 2)]
  probabilities = [];
  sumOfFitness = 0;
  previousProbabilities = 0;
  parents = [];

  for i in candidates:
    sumOfFitness = sumOfFitness + i['fitness']

  candidates.sort(key=sortByFitness)

 
  for cand in candidates:
    prob = previousProbabilities + (cand['fitness'] / sumOfFitness);
    previousProbabilities = prob;
    probabilities.append(prob);
  
  while len(parents)  < 2:
    number = random.randint(1, 10) /10
    found = False
    for y in range(0, len(probabilities)):
      if (number <= probabilities[y]) & (not found):
        parents.append(candidates[y])
        found = True
  
  return parents

In [ ]:
def mutate_activation(individue):
  new_attribute = activation[random.randint(0,3)] 
  while new_attribute == individue['activation']:
    new_attribute = activation[random.randint(0,3)] 
  individue['activation'] = new_attribute
  return individue

def mutate_learning_rate(individue):
  new_attribute = learning_rate[random.randint(0,2)] 
  while new_attribute == individue['learning_rate']:
    new_attribute = learning_rate[random.randint(0,2)] 
  individue['learning_rate'] = new_attribute
  return individue

def mutate_solver(individue):
  new_attribute = solver[random.randint(0,2)] 
  while new_attribute == individue['solver']:
    new_attribute = solver[random.randint(0,2)] 
  individue['solver'] = new_attribute
  return individue

def mutate_momentum(individue):
  new_attribute = random.randrange(0,1)
  while new_attribute == individue['momentum']:
    new_attribute = random.randrange(0,1)
  individue['momentum'] = new_attribute
  return individue

def mutate_batch_size(individue):
  new_attribute = random.randint(100,300)
  while new_attribute == individue['batch_size']:
    new_attribute = random.randint(100,300)
  individue['batch_size'] = new_attribute
  return individue

def mutate_shuffle(individue):
  individue['shuffle'] = not individue['shuffle']
  return individue

def mutate_warm_start(individue):
  individue['warm_start'] = not individue['warm_start']
  return individue

def mutate_nesterovs_momentum(individue):
  individue['nesterovs_momentum'] = not individue['nesterovs_momentum']
  return individue

# 'shuffle' : (random.randint(0,1) == 1),
#       'warm_start' : (random.randint(0,1) == 1),
#       'momentum' : random.randrange(0,1),
#       'nesterovs_momentum' : (random.randint(0,1) == 1),

def mutate(individue):
  mutations = [mutate_activation, mutate_learning_rate, mutate_solver,mutate_momentum,mutate_shuffle,mutate_warm_start,mutate_nesterovs_momentum,mutate_batch_size]
  prob = random.randint(0, 10)
  if prob >= 6:
    mutation1 = random.randint(0,len(mutations)-1)
    mutation2 = random.randint(0,len(mutations)-1)

    individue = mutations[mutation1](individue)
    individue = mutations[mutation2](individue)

  return individue

In [ ]:
def make_children(population):
  parents = choose_parents_by_roullete(population)
  
  for x in range(0, 2):
    child = {'activation': parents[random.randint(0,1)]['activation'],
            'hidden_layer_sizes': (parents[random.randint(0,1)]['hidden_layer_sizes'][0],parents[random.randint(0,1)]['hidden_layer_sizes'][1], parents[random.randint(0,1)]['hidden_layer_sizes'][2]),
            'learning_rate': parents[random.randint(0,1)]['learning_rate'],
            'max_iter': parents[random.randint(0,1)]['max_iter'],
            'solver': parents[random.randint(0,1)]['solver'],
            'shuffle': parents[random.randint(0,1)]['shuffle'],
            'warm_start': parents[random.randint(0,1)]['warm_start'],
            'momentum' : parents[random.randint(0,1)]['momentum'],
            'nesterovs_momentum' : parents[random.randint(0,1)]['nesterovs_momentum'],
            'batch_size' : parents[random.randint(0,1)]['batch_size'],
            'fitness': 0}
    

    child = mutate(child)
    child['fitness'] = calculate_fitness(child)
    population.append(child)
  
  population.sort(key=sortByFitness, reverse=True)
  return population[:POPULATION_SIZE]

#Main

In [ ]:
population = initiate_population(POPULATION_SIZE)
population.sort(key=sortByFitness, reverse=True)
print(population)
for x in range(0, GENERATIONS):
  print("generation: ")
  print(x)
  population = make_children(population)
  print(population)

print("Better was: ") 
print(population[0])
print("With a accuracy of: ")
print(population[0]['fitness'])

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (237) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (170) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571:

[{'activation': 'logistic', 'hidden_layer_sizes': (143, 68, 31), 'learning_rate': 'constant', 'max_iter': 144, 'solver': 'lbfgs', 'shuffle': True, 'warm_start': True, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 105, 'fitness': 0.71}, {'activation': 'relu', 'hidden_layer_sizes': (162, 66, 15), 'learning_rate': 'constant', 'max_iter': 156, 'solver': 'sgd', 'shuffle': True, 'warm_start': False, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 103, 'fitness': 0.7035}, {'activation': 'logistic', 'hidden_layer_sizes': (107, 86, 50), 'learning_rate': 'adaptive', 'max_iter': 189, 'solver': 'lbfgs', 'shuffle': True, 'warm_start': True, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 125, 'fitness': 0.702}, {'activation': 'tanh', 'hidden_layer_sizes': (145, 63, 36), 'learning_rate': 'constant', 'max_iter': 206, 'solver': 'sgd', 'shuffle': True, 'warm_start': False, 'momentum': 0, 'nesterovs_momentum': True, 'batch_size': 228, 'fitness': 0.699}, {'activation': 'id

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[{'activation': 'logistic', 'hidden_layer_sizes': (143, 68, 31), 'learning_rate': 'constant', 'max_iter': 144, 'solver': 'lbfgs', 'shuffle': True, 'warm_start': True, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 105, 'fitness': 0.71}, {'activation': 'relu', 'hidden_layer_sizes': (162, 66, 15), 'learning_rate': 'constant', 'max_iter': 156, 'solver': 'sgd', 'shuffle': True, 'warm_start': False, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 103, 'fitness': 0.7035}, {'activation': 'logistic', 'hidden_layer_sizes': (107, 86, 50), 'learning_rate': 'adaptive', 'max_iter': 189, 'solver': 'lbfgs', 'shuffle': True, 'warm_start': True, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 125, 'fitness': 0.702}, {'activation': 'tanh', 'hidden_layer_sizes': (145, 63, 36), 'learning_rate': 'constant', 'max_iter': 206, 'solver': 'sgd', 'shuffle': True, 'warm_start': False, 'momentum': 0, 'nesterovs_momentum': True, 'batch_size': 228, 'fitness': 0.699}, {'activation': 'id

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (129) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

[{'activation': 'logistic', 'hidden_layer_sizes': (143, 68, 31), 'learning_rate': 'constant', 'max_iter': 144, 'solver': 'lbfgs', 'shuffle': True, 'warm_start': True, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 105, 'fitness': 0.71}, {'activation': 'relu', 'hidden_layer_sizes': (162, 66, 15), 'learning_rate': 'constant', 'max_iter': 156, 'solver': 'sgd', 'shuffle': True, 'warm_start': False, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 103, 'fitness': 0.7035}, {'activation': 'logistic', 'hidden_layer_sizes': (107, 86, 50), 'learning_rate': 'adaptive', 'max_iter': 189, 'solver': 'lbfgs', 'shuffle': True, 'warm_start': True, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 125, 'fitness': 0.702}, {'activation': 'tanh', 'hidden_layer_sizes': (145, 63, 36), 'learning_rate': 'constant', 'max_iter': 206, 'solver': 'sgd', 'shuffle': True, 'warm_start': False, 'momentum': 0, 'nesterovs_momentum': True, 'batch_size': 228, 'fitness': 0.699}, {'activation': 'id

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (189) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (156) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[{'activation': 'logistic', 'hidden_layer_sizes': (143, 68, 31), 'learning_rate': 'constant', 'max_iter': 144, 'solver': 'lbfgs', 'shuffle': True, 'warm_start': True, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 105, 'fitness': 0.71}, {'activation': 'relu', 'hidden_layer_sizes': (162, 66, 15), 'learning_rate': 'constant', 'max_iter': 156, 'solver': 'sgd', 'shuffle': True, 'warm_start': False, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 103, 'fitness': 0.7035}, {'activation': 'logistic', 'hidden_layer_sizes': (107, 86, 50), 'learning_rate': 'adaptive', 'max_iter': 189, 'solver': 'lbfgs', 'shuffle': True, 'warm_start': True, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 125, 'fitness': 0.702}, {'activation': 'logistic', 'hidden_layer_sizes': (104, 86, 49), 'learning_rate': 'constant', 'max_iter': 189, 'solver': 'adam', 'shuffle': True, 'warm_start': True, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 198, 'fitness': 0.6995}, {'activation

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (156) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[{'activation': 'logistic', 'hidden_layer_sizes': (143, 68, 31), 'learning_rate': 'constant', 'max_iter': 144, 'solver': 'lbfgs', 'shuffle': True, 'warm_start': True, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 105, 'fitness': 0.71}, {'activation': 'logistic', 'hidden_layer_sizes': (162, 86, 50), 'learning_rate': 'constant', 'max_iter': 189, 'solver': 'lbfgs', 'shuffle': True, 'warm_start': False, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 103, 'fitness': 0.706}, {'activation': 'relu', 'hidden_layer_sizes': (162, 66, 15), 'learning_rate': 'constant', 'max_iter': 156, 'solver': 'sgd', 'shuffle': True, 'warm_start': False, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 103, 'fitness': 0.7035}, {'activation': 'logistic', 'hidden_layer_sizes': (107, 86, 50), 'learning_rate': 'adaptive', 'max_iter': 189, 'solver': 'lbfgs', 'shuffle': True, 'warm_start': True, 'momentum': 0, 'nesterovs_momentum': False, 'batch_size': 125, 'fitness': 0.702}, {'activatio